In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/conda/envs/elyza_llma/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

In [3]:
#model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
model_name = "elyza/ELYZA-japanese-Llama-2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.70it/s]


In [4]:
if torch.cuda.is_available():
    model = model.to("cuda")
    print("cuda", flush=True)

cuda


In [5]:
def make_prompt(text: str) -> str:
    return "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )

In [ ]:
text = (
    "以下の文を要約しなさい。"
    "要約文だけを出力しなさい。余計な文章はつけないでください。"
    "「クマが侵入し従業員が襲われた秋田市土崎港のスーパーで、クマを運び出した後に売り場を確認したところ精肉や総菜コーナーが荒らされていましたがパックに詰められた肉は食べられていないことが新たに分かりました。秋田市の農地森林整備課などによりますと、クマが侵入した秋田市土崎港のスーパーで２日ドローンを使って店内を調査しました。売り場の入口近くにあった花びんが割れていたほか売り場の精肉や総菜コーナーの商品が荒らされていたということです。一方で、パックに詰められた精肉を食べた形跡は確認されなかったということです。店に侵入したメスのクマは２日に市などが箱わなで捕獲しています。秋田市は店が設置した防犯カメラなどから、クマの侵入経路などの確認を続けています。」"
)

In [12]:
print(text)

以下の文内に統一した方が良い表記があれば、その表記を指摘しなさい。「稼動中のコンセントを抜かないでください。下記URLからご確認下さい。再度電源スイッチを押してください。」


In [13]:
prompt = make_prompt(text)

In [14]:
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=2000,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

In [15]:
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
print(output, flush=True)

承知しました。統一した表記があれば、その表記を指摘します。

統一した表記としては「稼働中のコンセントを抜かないでください。下記URLからご確認下さい。再度電源スイッチを押してください。」となります。

なお、「稼動中」と「稼働中」では意味が異なります。「稼動中」は動作をしている状態を表しますが、「稼働中」は動作をしている状態を表すとともに、その状態が正常かどうかを表すこともできます。
